In [1]:
import import_ipynb
import itertools

import random as rnd
import Cloud as cld
import numpy as np
import copy

from tqdm.notebook import tqdm
from pprint import pprint

importing Jupyter notebook from Cloud.ipynb


In [2]:
def shuffle(lst):
    arr = lst
    # Get length of List
    n = len(arr)
    #repeat the following for n number of times
    for i in range(n):
        #select an index randomly
        j = rnd.randint(0, n-1)
        #delete the element at that index.
        element=arr.pop(j)
        #now append that deleted element to the list
        arr.append(element)
    return arr

In [3]:
def array_split(lst, num_chunks):
    if num_chunks > len(lst):
        return lst
    assert(num_chunks > 0)
    # Chunk length
    i = len(lst) // num_chunks
    #print(i)
    l = [lst[k*i:(k+1)*i] for k in range(num_chunks-1)]
    l.append(lst[i*(num_chunks-1):])
    return l

#### Fitness function to calculate the fitness score of each gene
Calculated the max time that each server expends on its own workload.<br>
Need to take into account the speed of the processors

In [4]:
def fitness(chromosome):
    servers_fitness = []
    
    for server in chromosome.servers:
        core_num, core_speed, server_processes = server.cores, server.cpu_speed, server.workload
        execq = {c:[[],0] for c in range(core_num)}
        
        for proc in server_processes:
            key = min(execq, key=lambda p: execq[p][1])
            time_core = execq[key][1]
            time_core += proc.process_length
            execq[key][0].append(proc)
            execq[key][1] = time_core
            
        longest_time = max(execq, key=lambda p: execq[p][1]) #/ core_speed
        servers_fitness.append((server.name, execq[longest_time]))
    
    average_fitness = sum([pack[1][1] for pack in servers_fitness]) / len([pack[1][1] for pack in servers_fitness])
    return servers_fitness, average_fitness

#### Chromosome Class

In [5]:
class Chromosome:
    def __init__(self, servers, processes, individual=None, generation=0):
        self.servers = servers
        self.processes = processes
        self.detailed_fitness = fitness(self)
        self.fit = self.detailed_fitness[0]
        self.average_fitness = self.detailed_fitness[1]
        self.individual = individual
        self.generation = generation
    def __lt__(self, other):
         return self.average_fitness < other.average_fitness
    def __repr__(self):
        return "Average Chromosome fitness: {0}\n\tIndividual: {1}\n\tGENERATION: {2}".format(self.average_fitness, self.individual, self.generation)

In [6]:
num_servers = 20
cores = [16, 18, 20, 24, 48, 96]
cpu_speeds = [3, 4, 5, 10, 20]
rnd.seed(42)
servers = [cld.Server(name=x, cpu_speed=cpu_speeds[rnd.randint(0, 4)], cores=cores[rnd.randint(0, 4)]) for x in range(0, num_servers)]

In [7]:
num_processes = 4098
processes_lengths = [round(rnd.uniform(0.02, 1.000), 2) for x in range(0, num_processes)] 
# Fraction of billion of instructions. 1 = 1 billion instructions
processes = [cld.Process(x, processes_lengths[rnd.randint(0, len(processes_lengths)-1)]) for x in range(0, num_processes)]

In [8]:
def make_first_chromosome(servers, processes, individual):
    backup_servers = copy.deepcopy(servers)
    shuffled_processes = copy.deepcopy(processes)
    np.random.seed(individual)
    np.random.shuffle(shuffled_processes)
    split_processes = array_split(np.asarray(shuffled_processes), len(servers))
    #print(split_processes)
    for i in range(len(split_processes)):
        backup_servers[i].workload = split_processes[i]
        #print(backup_servers[i].workload)
    #print(servers[0].workload)
    chrom = Chromosome(backup_servers, processes, individual)
    del(backup_servers)
    del(shuffled_processes)
    return chrom

# Qualcosa in make_first_chromosome punta ai workload randomizzati, non a sè stesso. Porca madonna

In [9]:
chrom1 = make_first_chromosome(servers, processes, 0)

In [10]:
chrom2 = make_first_chromosome(servers, processes, 1)

chrom1.servers[0].workload

chrom2.servers[0].workload

In [11]:
chrom1 < chrom2, chrom1, chrom2

(False,
 Average Chromosome fitness: 5.202000000000001
 	Individual: 0
 	GENERATION: 0,
 Average Chromosome fitness: 5.156000000000001
 	Individual: 1
 	GENERATION: 0)

#### Function to create the first generation population

In [12]:
def make_first_gen(servers, processes, pop_size):
    population = []
    x = 0
    for i in tqdm(range(pop_size)):
        rnd.seed(i)#RandomState(i)
        #np.random.shuffle(processes)
        population.append(make_first_chromosome(servers, rnd.sample(copy.deepcopy(processes[:]), len(processes)), x))
        x += 1
    return sorted(population)

In [13]:
first_gen = make_first_gen(servers, processes, num_servers)

  0%|          | 0/20 [00:00<?, ?it/s]

In [14]:
first_gen

[Average Chromosome fitness: 5.193
 	Individual: 9
 	GENERATION: 0,
 Average Chromosome fitness: 5.214
 	Individual: 6
 	GENERATION: 0,
 Average Chromosome fitness: 5.2170000000000005
 	Individual: 19
 	GENERATION: 0,
 Average Chromosome fitness: 5.217999999999998
 	Individual: 14
 	GENERATION: 0,
 Average Chromosome fitness: 5.218
 	Individual: 8
 	GENERATION: 0,
 Average Chromosome fitness: 5.22
 	Individual: 4
 	GENERATION: 0,
 Average Chromosome fitness: 5.223000000000001
 	Individual: 12
 	GENERATION: 0,
 Average Chromosome fitness: 5.231
 	Individual: 10
 	GENERATION: 0,
 Average Chromosome fitness: 5.2335
 	Individual: 18
 	GENERATION: 0,
 Average Chromosome fitness: 5.234
 	Individual: 17
 	GENERATION: 0,
 Average Chromosome fitness: 5.237500000000001
 	Individual: 5
 	GENERATION: 0,
 Average Chromosome fitness: 5.2425
 	Individual: 15
 	GENERATION: 0,
 Average Chromosome fitness: 5.2455
 	Individual: 16
 	GENERATION: 0,
 Average Chromosome fitness: 5.251500000000002
 	Individu

#### Mutation Function
This function remixes the process list in a chromosome<br>
##### It appears to be useless by the nature of the crossover. Might want to look a bit more into it later

In [15]:
def mutation(chromosome):
    pass

#### Crossover Function
This function selects the best subjects from one generation and "breeds" them into the next. <br>
The two best subjects from the previous generation cross into the next. <br>
<br>
##### Crossbreed Logic
Compare the best performing workloads on each server. <br>
Take workload with best performance. <br>
If there are shared processess between servers:
1. Keep process on faster server;
2. Eliminate the process on slower server;
3. Place this process in a list;
4. Insert a random pending process from the list; **This is a mutation** 
5. Repeat redistribution 3. and 4. for num_children times.
6. The parents cross in the next generation pool.<br>

This should ensure a steadily improve each generation.

##### Breed logic [Randomized]
Compare the speed of the servers for each chromosome.
Reshape the workload to get only the fastest processes. <br>
1. Pick first server of chrom1 and random server of chrom2;
2. Pick fastest server of the two and merge workloads
3. Heaviest processes go to faster server
4. Repeat for every server of chrom1, exclude already picked servers of chrom3

This should ensure a steadily improve each generation.

In [16]:
# Breed function to generate one child from two chromosomes
def breed_rnd(chrom1, chrom2):
    already_picked_chrom2 = []
    merged_workloads = np.asarray([])
    new_servers = []
    for server in chrom1.servers:
        already_picked_chrom2.append(server)
        rnd_pick = rnd.randint(0, len(chrom2.servers)-1)
        #print(type(server.workload))
        merged_workloads = np.append(merged_workloads, server.workload)#.tolist()
        while chrom2.servers[rnd_pick] in already_picked_chrom2:
            #print("Error")
            rnd_pick = rnd.randint(0, len(chrom2.servers)-1)
        server2 = chrom2.servers[rnd_pick]
        already_picked_chrom2.append(chrom2.servers[rnd_pick])
        
        merged_workloads = np.append(merged_workloads, server2.workload)#.tolist()
        
        # Pick fastest server of the two and reset its workload
        fastest = max((server, server2), key=lambda x: x.cpu_speed * x.cores)
        slowest = min((server, server2), key=lambda x: x.cpu_speed * x.cores)
        worksize = len(fastest.workload)
        fastest.workload = []
        
        # Sort merged workloads to pick heaviest first
        merged_workloads = sorted(merged_workloads, key=lambda x: x.process_length, reverse=True)
        fastest.workloads = merged_workloads[worksize:]
        merged_workloads = merged_workloads[:worksize]
        
        # TODO: CREATE CHROMOSOME, MAKE SURE NOT TO ADD DUPLICATES
        new_servers.append(fastest)
        new_servers.append(slowest)
    return new_servers
    #return #sorted(already_picked_chrom2, key=lambda x: x.name)
        

In [17]:
max((chrom1.servers[0], chrom2.servers[2]), key=lambda x: x.cpu_speed * x.cores)

Server: 2
	Number of Cores: 18
	Core Speed: 4

In [18]:
breed_rnd(chrom1, chrom2)

[Server: 16
 	Number of Cores: 18
 	Core Speed: 4,
 Server: 0
 	Number of Cores: 16
 	Core Speed: 3,
 Server: 1
 	Number of Cores: 18
 	Core Speed: 5,
 Server: 1
 	Number of Cores: 18
 	Core Speed: 5,
 Server: 2
 	Number of Cores: 18
 	Core Speed: 4,
 Server: 19
 	Number of Cores: 20
 	Core Speed: 3,
 Server: 5
 	Number of Cores: 16
 	Core Speed: 10,
 Server: 3
 	Number of Cores: 48
 	Core Speed: 3,
 Server: 4
 	Number of Cores: 48
 	Core Speed: 3,
 Server: 13
 	Number of Cores: 16
 	Core Speed: 5,
 Server: 12
 	Number of Cores: 48
 	Core Speed: 10,
 Server: 5
 	Number of Cores: 16
 	Core Speed: 10,
 Server: 10
 	Number of Cores: 48
 	Core Speed: 4,
 Server: 6
 	Number of Cores: 16
 	Core Speed: 3,
 Server: 7
 	Number of Cores: 18
 	Core Speed: 4,
 Server: 6
 	Number of Cores: 16
 	Core Speed: 3,
 Server: 8
 	Number of Cores: 48
 	Core Speed: 20,
 Server: 0
 	Number of Cores: 16
 	Core Speed: 3,
 Server: 9
 	Number of Cores: 48
 	Core Speed: 3,
 Server: 14
 	Number of Cores: 24
 	Core 

In [19]:
sorted(breed_rnd(chrom1, chrom2), key=lambda x: x.pid)

AttributeError: 'Server' object has no attribute 'pid'

In [230]:
def crossover(generation, num_children, num_surv=2):
    # Select best specimen
    survivors = generation[:2]#[:num_surv]
    print(survivors)
    # Num_surv is variable but let's assume it is always two
    father, mother = survivors[0], survivors[1]
    # Here are placed the best servers with no workload in common
    absolute_gene_expression = []
    #for i in range(len(father.workload)):
        
    # Check fittest servers between the two chromosomes
        # Need to calculate the fitness of each server

In [231]:
crossover(first_gen, 3)

[Average Chromosome fitness: 5.199
	Individual: 7
	GENERATION: 0, Average Chromosome fitness: 5.216000000000001
	Individual: 14
	GENERATION: 0]


In [213]:
a, b = first_gen[0], first_gen[4]
server_fitness(a.__dict__["servers"][0]) == server_fitness(b.__dict__["servers"][0])#.workload

[PID: 23937
 	Length of process: 0.3 instructions
 PID: 18078
 	Length of process: 0.17 instructions
 PID: 26913
 	Length of process: 0.23 instructions ...
 PID: 12210
 	Length of process: 0.73 instructions
 PID: 4394
 	Length of process: 0.24 instructions
 PID: 12619
 	Length of process: 0.47 instructions]
[PID: 23937
 	Length of process: 0.3 instructions
 PID: 18078
 	Length of process: 0.17 instructions
 PID: 26913
 	Length of process: 0.23 instructions ...
 PID: 12210
 	Length of process: 0.73 instructions
 PID: 4394
 	Length of process: 0.24 instructions
 PID: 12619
 	Length of process: 0.47 instructions]


True

array([PID: 23937
       	Length of process: 0.3 instructions,
       PID: 18078
       	Length of process: 0.17 instructions,
       PID: 26913
       	Length of process: 0.23 instructions, ...,
       PID: 12210
       	Length of process: 0.73 instructions,
       PID: 4394
       	Length of process: 0.24 instructions,
       PID: 12619
       	Length of process: 0.47 instructions], dtype=object)

In [216]:
a.__dict__["servers"][0].workload==b.__dict__["servers"][0].workload

array([ True,  True,  True, ...,  True,  True,  True])